In [ ]:
from dotenv import load_dotenv
import numpy as np
import os
import sys
import pickle
sys.path.append('/Users/david/Desktop/FinetuneEmbed')

from tqdm import tqdm
from openai import OpenAI

os.chdir('/Users/david/Desktop/FinetuneEmbed')
print(os.getcwd())

# set up the OpenAI API key CONFIDENTIAL
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

with open('./data/embeddings/gene_name_list.pickle', 'rb') as f:
    loaded_list = pickle.load(f)


/Users/david/Desktop/FinetuneEmbed


In [2]:
loaded_list = loaded_list[0:10]
print(loaded_list)

['AC083867.4', '', 'RNF187', 'ATF7IP2', 'RP4-639J15.2', 'RP11-137J7.2', 'PALB2', 'FLG2', 'AC067742.1', 'RNA5SP79']


In [3]:
# initiate openai client
client = OpenAI(api_key=openai_api_key)

# use the embedding model text-embedding-ada-002
def get_gpt_embedding(client, text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [4]:
gpt_gene_name_to_embedding_clean_text = {}
GPT_DIM = 3072 # fix GPT embeddings
for name in tqdm(loaded_list):
    if name not in gpt_gene_name_to_embedding_clean_text:
        # print('key', key)
        if name == '': 
            # if the dictionary does not have information about a gene
            gpt_gene_name_to_embedding_clean_text[name] = np.zeros(GPT_DIM) # it's hard coded to be 0
        else:
            gpt_gene_name_to_embedding_clean_text[name] = get_gpt_embedding(client, name) 


100%|██████████| 10/10 [00:07<00:00,  1.37it/s]


In [6]:

# Save
with open('./data/embeddings/GeneName_embed.pickle', 'wb') as f:
    pickle.dump(gpt_gene_name_to_embedding_clean_text, f)